# SVC - No multi glitch

**N.B.** SVC require normalized data; nevertheless, in this case _we can't normalize the features_, otherwise we would lose all the information on the peaks!! In fact, if we do not normalize the data, the train score is around $0.98$ or $0.99$; on the contrary, if we normalize the data, the train score is around $0.70$.

In [ ]:
%matplotlib inline

# Scientific computing
import numpy as np
import pandas as pd
from scipy import interp

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#sns.set_context('paper')

# Machine Learning
# Model
from sklearn.svm import SVC
# Ensemble model
from sklearn.ensemble import BaggingClassifier
# Splitter Classes
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
# Splitter Functions
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
# Model validation
from sklearn.model_selection import learning_curve
# Training metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# Other
import requests
import threading

# Useful functions

Function `print_confusion_matrix`

In [ ]:
def print_confusion_matrix(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(str(x)) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    
    # Begin CHANGES
    fst_empty_cell = (columnwidth-3)//2 * " " + "t\p" + (columnwidth-3)//2 * " "
    
    if len(fst_empty_cell) < len(empty_cell):
        fst_empty_cell = " " * (len(empty_cell) - len(fst_empty_cell)) + fst_empty_cell
    # Print header
    print("    " + fst_empty_cell, end=" ")
    # End CHANGES
    
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
        
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

Function `plot_confusion_matrix`

In [ ]:
def plot_confusion_matrix(cm, labels, title, filename, normalize=True, save=True):
    # Create DataFrame
    df_cm = pd.DataFrame(cm, columns=labels, index=labels)
    df_cm.index.name = 'True label'
    df_cm.columns.name = 'Predicted label'
    
    # Normalize
    if normalize:
        df_cm = df_cm.div(df_cm.sum(axis=1), axis=0).round(decimals=2)
    
    # Plot
    fig, ax = plt.subplots()
    ax = sns.heatmap(df_cm, cmap='Blues', annot=True)
    axlim = ax.get_ylim()
    ax.set_ylim(axlim[0] + 0.5, axlim[1] - 0.5)
    if normalize:
        ax.set_title(title + ' (normalized)')
    else:
        ax.set_title(title)
    
    if save == True:
        fig.savefig(filename)

Function `plot_learning_curve`

In [ ]:
def plot_learning_curve(estimator, X, y, title, filename, ylim=None, xscale=None, cv=None,
                        n_jobs=-1, n_curve_steps=20, save=True):
    train_sizes = np.linspace(.1, 1.0, n_curve_steps)
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    if xscale is not None:
        plt.xscale(xscale)
    plt.xlabel('Training dataset size')
    plt.ylabel('Score')
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color='r')
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color='g')
    plt.plot(train_sizes, train_scores_mean, linestyle='--', color='r',
             label='Training score')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='g',
             label='Cross-validation score')

    plt.legend(loc='best')
    
    if save == True:
        plt.savefig(filename)

Function `plot_roc_curve`

In [ ]:
def plot_roc_curve(estimator, X, y, cv, title, filename, save=True):
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    i = 0
    for train, test in cv.split(X, y):
        probas_ = estimator.fit(X[train], y[train]).predict_proba(X[test])
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3,
                 label='ROC fold %d (AUC = %0.3f)' % (i, roc_auc))
        i += 1

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.3f $\pm$ %0.3f)' % (mean_auc, std_auc),
             lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                     label=r'$\pm$ 1 std. dev.')

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    
    if save == True:
        plt.savefig(filename)

# Preparation

Load data and target from `classification/ris/OUT-classified-merged.h5` and load into numpy arrays.
- **Label `0` = NO GLITCH**
- **Label `1` = GLITCH**

In [ ]:
first_cycle = True
with pd.HDFStore('../../classification/ris/OUT-classified-merged.h5', mode='r') as in_data:
    for group in ['GLITCH', 'NO_GLITCH']:
        if first_cycle == True:
            data = np.array(in_data[group].to_numpy())
            if group == 'GLITCH':
                target = np.ones(len(data))
            elif group == 'NO_GLITCH':
                target = np.zeros(len(data))
            else:
                print("ERROR.")
            first_cycle = False
        else:
            data = np.concatenate((data, in_data[group].to_numpy()))
            if group == 'GLITCH':
                target = np.concatenate((target, np.ones(len(in_data[group].to_numpy()))))
            elif group == 'NO_GLITCH':
                target = np.concatenate((target, np.zeros(len(in_data[group].to_numpy()))))
            else:
                print("ERROR.")

# Standard training

Initialize best hyper-parameters founded.

In [ ]:
best_kernel = 'rbf'
best_gamma = 0.0145
best_C = 0.8

## Cross validation

Use k-fold to make a cross validation of the model.

In [ ]:
# Multithreading
maxthreads = 8
sema = threading.Semaphore(value=maxthreads)
c = threading.Condition()

# k-fold parameters
n_splits = 5
n_repeats = 10

# Multithread function
def thread_function(train_index, test_index):
    # Acquire a semaphore slot
    sema.acquire()
    # Set global variables
    global data
    global target
    global scores
    global best_kernel
    global best_gamma
    global best_C
    # Load training and testing data
    c.acquire()
    clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    c.notify_all()
    c.release()
    # Fit the model
    clf.fit(X_train, y_train)
    train_score = clf.score(X_test, y_test)
    # Save the score
    c.acquire()
    scores = np.append(scores, train_score)
    c.notify_all()
    c.release()
    # Release the semaphore slot
    sema.release()

# Stratified k-fold
rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=None)
scores = np.array([])
threads = []
# Make k-fold CV
for train_index, test_index in rskf.split(data, target):
    thread = threading.Thread(target=thread_function, args=(train_index, test_index))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
    
# Print final score
print('Score:', scores.mean(), '+-', scores.std())

## Confusion matrix, learning curve and ROC curve

### Confusion matrix

In [ ]:
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=14, stratify=target)
# Train and predict
y_pred = clf.fit(X_train, y_train).predict(X_test)

# Compute confusion matrix
labels = [0., 1.]
labels_text = ['no glitch', 'glitch']
cm = confusion_matrix(y_test, y_pred, labels=labels)
# Print confusion matrix
print('Text confusion matrix (without normalization)\n')
print_confusion_matrix(cm, labels_text)
print('\n\nGraphic confusion matrix (normalized)\n')
plot_confusion_matrix(cm, labels_text, 'Confusion matrix (no multi glitch)', 'ris/plots/nmg-confusion_matrix.pdf', normalize=True, save=False)

### Learning curve

In [ ]:
# Classifier and cross-validation method
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)
cv = ShuffleSplit(n_splits=100, test_size=0.3, random_state=0)

# Plot learning curve
plot_learning_curve(clf, data, target, 'Learning curve (no multi glitch)', 'ris/plots/nmg-learning_curve.pdf', cv=cv, save=False)

### ROC curve

In [ ]:
# Classifier and cross-validation method
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C, probability=True)
cv = StratifiedKFold(n_splits=5)

# Plot ROC curve
plot_roc_curve(clf, data, target, cv, 'ROC curve (no multi glitch)', 'ris/plots/nmg-roc_curve.pdf')

# Sorted data

Glitches data have a peak, but it is in a random point of the plot. We can help the model by sorting data values: the new features queue will be about the same for all the data, while the first features will became discriminant.

In [ ]:
data_s = np.sort(data, axis=1)

Initialize best hyper-parameters founded.

In [ ]:
best_kernel = 'linear'
best_C = 0.8

## Cross validation

Use k-fold to make a cross validation of the model.

In [ ]:
# Multithreading
maxthreads = 8
sema = threading.Semaphore(value=maxthreads)
c = threading.Condition()

# k-fold parameters
n_splits = 5
n_repeats = 10

# Multithread function
def thread_function(train_index, test_index):
    # Acquire a semaphore slot
    sema.acquire()
    # Set global variables
    global data
    global target
    global scores
    global best_kernel
    global best_gamma
    global best_C
    # Load training and testing data
    c.acquire()
    clf = SVC(kernel=best_kernel, C=best_C)
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    c.notify_all()
    c.release()
    # Fit the model
    clf.fit(X_train, y_train)
    train_score = clf.score(X_test, y_test)
    # Save the score
    c.acquire()
    scores = np.append(scores, train_score)
    c.notify_all()
    c.release()
    # Release the semaphore slot
    sema.release()

# Stratified k-fold
rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=None)
scores = np.array([])
threads = []
# Make k-fold CV
for train_index, test_index in rskf.split(data, target):
    thread = threading.Thread(target=thread_function, args=(train_index, test_index))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
    
# Print final score
print('Score:', scores.mean(), '+-', scores.std())

## Confusion matrix, learning curve and ROC curve

### Confusion matrix

In [ ]:
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=14, stratify=target)
# Train and predict
y_pred = clf.fit(X_train, y_train).predict(X_test)

# Compute confusion matrix
labels = [0., 1.]
labels_text = ['no glitch', 'glitch']
cm = confusion_matrix(y_test, y_pred, labels=labels)
# Print confusion matrix
print('Text confusion matrix (without normalization)\n')
print_confusion_matrix(cm, labels_text)
print('\n\nGraphic confusion matrix (normalized)\n')
plot_confusion_matrix(cm, labels_text, 'Confusion matrix (no multi glitch) (sorted)', 'ris/plots/nmg_s-confusion_matrix.pdf', normalize=True, save=False)

### Learning curve

In [ ]:
# Classifier and cross-validation method
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)
cv = ShuffleSplit(n_splits=100, test_size=0.3, random_state=0)

# Plot learning curve
plot_learning_curve(clf, data, target, 'Learning curve (no multi glitch) (sorted)', 'ris/plots/nmg_s-learning_curve.pdf', cv=cv, save=False)

### ROC curve

In [ ]:
# Classifier and cross-validation method
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C, probability=True)
cv = StratifiedKFold(n_splits=5)

# Plot ROC curve
plot_roc_curve(clf, data, target, cv, 'ROC curve (no multi glitch) (sorted)', 'ris/plots/nmg_s-roc_curve.pdf')

# Sorted data (and absolute value)

Glitches data have a peak, but it is in a random point of the plot. We can help the model by sorting data values: the new features queue will be about the same for all the data, while the first features will became discriminant.

In [ ]:
data_sa = np.abs(np.sort(data, axis=1))

## Non-normalized data

Initialize best hyper-parameters founded.

In [ ]:
best_kernel = 'rbf'
best_gamma = 0.0145
best_C = 0.8

### Cross validation

Use k-fold to make a cross validation of the model.

In [ ]:
# Multithreading
maxthreads = 8
sema = threading.Semaphore(value=maxthreads)
c = threading.Condition()

# k-fold parameters
n_splits = 5
n_repeats = 10

# Multithread function
def thread_function(train_index, test_index):
    # Acquire a semaphore slot
    sema.acquire()
    # Set global variables
    global data_sa
    global target
    global scores
    global best_kernel
    global best_gamma
    global best_C
    # Load training and testing data
    c.acquire()
    clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)
    X_train, X_test = data_sa[train_index], data_sa[test_index]
    y_train, y_test = target[train_index], target[test_index]
    c.notify_all()
    c.release()
    # Fit the model
    clf.fit(X_train, y_train)
    train_score = clf.score(X_test, y_test)
    # Save the score
    c.acquire()
    scores = np.append(scores, train_score)
    c.notify_all()
    c.release()
    # Release the semaphore slot
    sema.release()

# Stratified k-fold
means = np.array([])
for i in range(n_repeats):
    rskf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=None)
    scores = np.array([])
    threads = []
    # Make k-fold CV
    for train_index, test_index in rskf.split(data_sa, target):
        thread = threading.Thread(target=thread_function, args=(train_index, test_index))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    means = np.append(means, scores.mean())
    
# Print final score
print('Score:', means.mean(), '+-', means.std() / np.sqrt(n_splits))

### Confusion matrix, learning curve and ROC curve

#### Confusion matrix

In [ ]:
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(data_sa, target, test_size=0.3, random_state=14, stratify=target)
# Train and predict
y_pred = clf.fit(X_train, y_train).predict(X_test)

# Compute confusion matrix
labels = [0., 1.]
labels_text = ['no glitch', 'glitch']
cm = confusion_matrix(y_test, y_pred, labels=labels)
# Print confusion matrix
print('Text confusion matrix (without normalization)\n')
print_confusion_matrix(cm, labels_text)
print('\n\nGraphic confusion matrix (normalized)\n')
plot_confusion_matrix(cm, labels_text, 'Confusion matrix (no multi glitch)', 'ris/plots/confusion_matrix.pdf', normalize=True)

#### Learning curve

In [ ]:
# Classifier and cross-validation method
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)
cv = ShuffleSplit(n_splits=100, test_size=0.3, random_state=0)

# Plot learning curve
plot_learning_curve(clf, data_sa, target, 'Learning curve (no multi glitch)', 'ris/plots/learning_curve.pdf', cv=cv, n_jobs=-1)

#### ROC curve

In [ ]:
# Classifier and cross-validation method
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C, probability=True)
cv = StratifiedKFold(n_splits=5)

# Plot ROC curve
plot_roc_curve(clf, data_sa, target, cv, 'ROC curve (no multi glitch)', 'ris/plots/roc_curve.pdf')

# Data augmentation

Data augmentation is a strategy that increase the diversity of data available for training models, without actually collecting new data. The data augmentation techniques used in this situation are vertical flipping and translation.

**N.B.: the training time is extremely big!**

## Training

Due to the long training time, this part will be made in another script on `belen`, called `score_algorithms.py` .

### Simple training

In [ ]:
# See score_algorithms.py

### Bagging Classifier training

A Bagging classifier is an ensemble meta-estimator that fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction.

In [ ]:
# See score_algorithms.py

## Confusion matrix, learning curve and ROC curve

Due to the long training time, this part will be made in another script on `belen`, called `data_augmentation_plots.py` .

### Confusion matrix

In [ ]:
# See data_augmentation_plots.py

### Learning curve

In [ ]:
# See data_augmentation_plots.py

### ROC curve

In [ ]:
# See data_augmentation_plots.py

# YES MULTI GLITCH

## Preparation

Load data and target from `classification/ris/OUT-classified-merged.h5` and load into numpy arrays.

**Label `0` = NO GLITCH**

**Label `1` = GLITCH and MULTI GLITCH**

In [ ]:
with pd.HDFStore('../../classification/ris/OUT-classified-merged.h5', mode='r') as in_data:
    data = np.concatenate((data, in_data['MULTI_GLITCH'].to_numpy()))
    target = np.concatenate((target, np.ones(len(in_data['MULTI_GLITCH'].to_numpy()))))

## Best training

Initialize best hyper-parameters founded.

In [ ]:
best_kernel = 'rbf'
best_gamma = 0.0151
best_C = 1.45

### Cross validation

Use k-fold to make a cross validation of the model.

In [ ]:
# Multithreading
maxthreads = 8
sema = threading.Semaphore(value=maxthreads)
c = threading.Condition()

# k-fold parameters
n_splits = 5
n_repeats = 10

# Multithread function
def thread_function(train_index, test_index):
    # Acquire a semaphore slot
    sema.acquire()
    # Set global variables
    global data
    global target
    global scores
    global best_kernel
    global best_gamma
    global best_C
    # Load training and testing data
    c.acquire()
    clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    c.notify_all()
    c.release()
    # Fit the model
    clf.fit(X_train, y_train)
    train_score = clf.score(X_test, y_test)
    # Save the score
    c.acquire()
    scores = np.append(scores, train_score)
    c.notify_all()
    c.release()
    # Release the semaphore slot
    sema.release()

# Stratified k-fold
means = np.array([])
for i in range(n_repeats):
    rskf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=None)
    scores = np.array([])
    threads = []
    # Make k-fold CV
    for train_index, test_index in rskf.split(data, target):
        thread = threading.Thread(target=thread_function, args=(train_index, test_index))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    means = np.append(means, scores.mean())
    
# Print final score
print('Score:', means.mean(), '+-', means.std() / np.sqrt(n_splits))

### Confusion matrix, learning curve and ROC curve

#### Confusion matrix

In [ ]:
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=14, stratify=target)
# Train and predict
y_pred = clf.fit(X_train, y_train).predict(X_test)

# Compute confusion matrix
labels = [0., 1.]
labels_text = ['no glitch', 'glitch']
cm = confusion_matrix(y_test, y_pred, labels=labels)
# Print confusion matrix
print('Text confusion matrix (without normalization)\n')
print_confusion_matrix(cm, labels_text)
print('\n\nGraphic confusion matrix (normalized)\n')
plot_confusion_matrix(cm, labels_text, 'Confusion matrix (no multi glitch)', 'ris/plots/confusion_matrix.pdf', normalize=True)

#### Learning curve

In [ ]:
# Classifier and cross-validation method
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)
cv = ShuffleSplit(n_splits=100, test_size=0.3, random_state=0)

# Plot learning curve
plot_learning_curve(clf, data, target, 'Learning curve (no multi glitch)', 'ris/plots/learning_curve.pdf', cv=cv, n_jobs=-1)

#### ROC curve

In [ ]:
# Classifier and cross-validation method
clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C, probability=True)
cv = StratifiedKFold(n_splits=5)

# Plot ROC curve
plot_roc_curve(clf, data, target, cv, 'ROC curve (no multi glitch)', 'ris/plots/roc_curve.pdf')

## Data augmentation

Data augmentation is a strategy that increase the diversity of data available for training models, without actually collecting new data. The data augmentation techniques used in this situation are vertical flipping and translation.

### Preparation

In [ ]:
data_aug, target_aug = data, target
data_aug = np.concatenate((data_aug, -data))
target_aug = np.concatenate((target_aug, target))

for i in range(1,100):
    data_aug = np.concatenate((data_aug, np.roll(data, i, axis=1)))
    data_aug = np.concatenate((data_aug, -np.roll(data, i, axis=1)))
    target_aug = np.concatenate((target_aug, target))
    target_aug = np.concatenate((target_aug, target))

### Simple training

In [ ]:
maxthreads = 4
sema = threading.Semaphore(value=maxthreads)
c = threading.Condition()

def thread_function(train_index, test_index):
    # Acquire a semaphore slot
    sema.acquire()
    # Set global variables
    global data_aug
    global target_aug
    global scores
    global best_kernel
    global best_gamma
    global best_C
    # Load training and testing data
    c.acquire()
    clf = SVC(kernel=best_kernel, gamma=best_gamma, C=best_C)
    X_train, X_test = data_aug[train_index], data_aug[test_index]
    y_train, y_test = target_aug[train_index], target_aug[test_index]
    c.notify_all()
    c.release()
    # Fit the model
    clf.fit(X_train, y_train)
    # Save the score
    c.acquire()
    scores = np.append(scores, clf.score(X_test, y_test))
    c.notify_all()
    c.release()
    # Release the semaphore slot
    sema.release()

    
## K-FOLD

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None)
scores = np.array([])
threads = []
# Make k-fold CV
for train_index, test_index in rkf.split(data_aug, target_aug):
    thread = threading.Thread(target=thread_function, args=(train_index, test_index))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
# Print final score
print('Average score (k-fold):', scores.mean(), '+-', scores.std())


## STRATIFIED K-FOLD

rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=None)
scores = np.array([])
threads = []
# Make k-fold CV
for train_index, test_index in rskf.split(data_aug, target_aug):
    thread = threading.Thread(target=thread_function, args=(train_index, test_index))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
# Print final score
print('Average score (Stratified k-fold):', scores.mean(), '+-', scores.std())

In [ ]:
telegram_bot_id = toml.load('../telegram_bot_id.toml')
params = {'chat_id': telegram_bot_id['chat_id'], 'text': '[python] End k-fold validation.'}
requests.post('https://api.telegram.org/' + telegram_bot_id['bot_id'] + '/sendMessage', params=params)

### Bagging Classifier training

A Bagging classifier is an ensemble meta-estimator that fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction.

In [ ]:
telegram_bot_id = toml.load('../telegram_bot_id.toml')
params = {'chat_id': telegram_bot_id['chat_id'], 'text': '[python] Start Bagging Classifier k-fold validation.'}
requests.post('https://api.telegram.org/' + telegram_bot_id['bot_id'] + '/sendMessage', params=params)

In [ ]:
n_estimators = 10
clf = BaggingClassifier(SVC(kernel=best_kernel, gamma=best_gamma, C=best_C), n_estimators=n_estimators, max_samples=1./n_estimators, n_jobs=-1)


## K-FOLD

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None)
scores_rkf = np.array([])
# Make k-fold CV
for train_index, test_index in rkf.split(data_aug, target_aug):
    X_train, X_test = data_aug[train_index], data_aug[test_index]
    y_train, y_test = target_aug[train_index], target_aug[test_index]
    clf.fit(X_train, y_train)
    scores_rkf = np.append(scores_rkf, clf.score(X_test, y_test))
# Print final score
print('Average score (k-fold):', scores_rkf.mean(), '+-', scores_rkf.std())


## STRATIFIED K-FOLD

rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=None)
scores_rskf = np.array([])
# Make k-fold CV
for train_index, test_index in rskf.split(data_aug, target_aug):
    X_train, X_test = data_aug[train_index], data_aug[test_index]
    y_train, y_test = target_aug[train_index], target_aug[test_index]
    clf.fit(X_train, y_train)
    scores_rskf = np.append(scores_rskf, clf.score(X_test, y_test))
# Print final score
print('Average score (Stratified k-fold):', scores_rskf.mean(), '+-', scores_rskf.std())

In [ ]:
telegram_bot_id = toml.load('../telegram_bot_id.toml')
params = {'chat_id': telegram_bot_id['chat_id'], 'text': '[python] End Bagging Classifier k-fold validation.'}
requests.post('https://api.telegram.org/' + telegram_bot_id['bot_id'] + '/sendMessage', params=params)

In [ ]:
telegram_bot_id = toml.load('../telegram_bot_id.toml')
params = {'chat_id': telegram_bot_id['chat_id'], 'text': '[python] End data augmentation part.'}
requests.post('https://api.telegram.org/' + telegram_bot_id['bot_id'] + '/sendMessage', params=params)

In [ ]:
telegram_bot_id = toml.load('../telegram_bot_id.toml')
params = {'chat_id': telegram_bot_id['chat_id'], 'text': '[python] End yes multi glitch part.'}
requests.post('https://api.telegram.org/' + telegram_bot_id['bot_id'] + '/sendMessage', params=params)